In [22]:
#hide
#default_exp tools.query
from nbdev.showdoc import *
from dsblocks.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
#tst = TestRunner (targets=['dummy'])
tst = TestRunner (targets=[])

# Query

> Shows results for queried experiments.

In [23]:
#export
import warnings
warnings.filterwarnings('ignore')

import argparse
import sys
sys.path.append('.')
from collections import namedtuple
from IPython.display import display
import pandas as pd

# hpsearch api
import hpsearch.utils.experiment_utils as ut
import hpsearch.config.hp_defaults as dflt
from hpsearch.config.hpconfig import get_experiment_manager, get_em_args, add_em_args

In [24]:
#for tests
import pytest
import os

import hpsearch.config.hp_defaults as dflt
from hpsearch.examples.dummy_experiment_manager import generate_data

## query

In [25]:
# export
def query (pv={}, pf={}, pall=[], pexact=False, folder=None, 
           metric=None, experiments=None, runs=None, op=None, stats=['mean'], 
           results=0, other_parameters=False, **kwargs):
    
    result_query = ut.query(folder_experiments=folder, score_name=metric, experiments=experiments,
                            run_number=runs, parameters_fixed=pf, parameters_variable=pv, 
                            parameters_all=pall, exact_match=pexact, ascending=op=='min', 
                            stats=stats, min_results=results, 
                            query_other_parameters=other_parameters, **kwargs)
    
    return result_query 

## do_query_and_show

In [26]:
#export
def do_query_and_show (pall=[], best=None, compact=0, exact=False, experiments=None, pf={}, last=None, 
                       other_parameters=False, input_range=None, results=0, 
                       round=2, runs=None, show=False, stats=['mean'], pv={},
                       sort=None, display_all_columns=False, col_width=None, **kwargs):
    em_args = get_em_args (kwargs)
    em = get_experiment_manager (**em_args)
    kwargs = {k: kwargs[k] for k in kwargs if k not in em_args}
    folder, metric, op = em.folder, em.key_score, em.op
        
    df = query (pv=pv, pf=pf, pall=pall, pexact=exact, folder=em.folder, 
               metric=em.key_score, experiments=experiments, runs=runs, op=em.op, stats=stats, 
               results=results, other_parameters=other_parameters, **kwargs)
    df = ut.replace_with_default_values (df)
    if sort is not None:
        stats_cols = df[dflt.stats_col].columns.get_level_values(1)
        pars_cols = df[dflt.parameters_col].columns.get_level_values(0)
        if sort in stats_cols:
            score_name_sort = df[dflt.stats_col].columns.get_level_values(0).unique()
            if len(score_name_sort)>1: print (f'sorting using first score_name from {score_name_sort}')
            score_name_sort = score_name_sort[0]
            sort_col = (dflt.stats_col, score_name_sort, sort)
        elif sort in pars_col:
            sort_col = (dflt.parameters_col, sort, '')
        else:
            raise ValueError (f'sort must be in {stats_cols.tolist()+pars_cols.tolist()}')
        df = df.sort_values(by=sort_col, ascending=(em.op=='min'))
    if experiments is None:
        experiments = []
    if last is not None:
        experiments += range(df.index.max()-last+1, df.index.max()+1)
    if best is not None:
        experiments += list(df.index[:best])
    if input_range is not None:
        assert len(input_range) == 2
        experiments += range(input_range[0], input_range[1])
    if len(experiments) > 0: 
        df = df.loc[[x for x in df.index if x in experiments]]
    
    if col_width is not None:
        pd.set_option('max_colwidth', col_width)
    
    if (round is not None) and (round != 0):
        stats_col = pd.MultiIndex.from_product ([[dflt.stats_col], 
                                                 df[dflt.stats_col].columns.get_level_values(0).unique(),
                                                 stats])
        df[stats_col] = df[stats_col].round(round)
    if display_all_columns:
        display (df)
    
    print (f'experiments: {list(df.index)}')
    print (f'min experiment #: {df.index.min()}, max experiment #: {df.index.max()}')

    print ('result of query:')
    _, df2 = ut.get_parameters_unique(df)
    #df2.index.name = 'experiment #'
    if compact > 0:
        prev_cols = df2.columns.copy()
        df2, dict_rename = ut.compact_parameters (df2, compact)
        for k, kor in zip(df2.columns, prev_cols):
            print (f'{k} => {kor}')
    display (df2)
            
    if show:
        import hpsearch.visualization.plot_visdom as pv
        pv.plot_multiple_histories(df.index, folder=em.folder,metrics=em.key_score, parameters=None)
    return df2

### Run simple query

Run query without any condition, retrieving all the data

In [27]:
#export tests.tools.test_query
def test_do_query_and_show ():
    path_results = 'do_query_and_show'
    em = generate_data (path_results)
    
    df=do_query_and_show (manager_path=em.manager_path)
    assert sorted(os.listdir (f'test_{path_results}/default/managers'))==['fields', 'info', 'logs.txt', 'whole']
    assert sorted(os.listdir (f'test_{path_results}/default/managers/whole'))==['DummyExperimentManager-default.pk', 'last.pk']
    par = lambda parameter: (dflt.parameters_col, parameter, '')
    assert (df[par('epochs')] == [15,30,5,15,30,15,5,30,5]).all()
    assert (df[par('offset')] == [.6,.6,.6,.3,.3,.1,.3,.1,.1]).all()
    assert (df[(dflt.stats_col, 'validation_accuracy', 'mean')] == [0.97, 0.89, 0.81, 0.8 , 0.65, 0.55, 0.46, 0.44, 0.19]).all()

    em.remove_previous_experiments (parent=True)

In [28]:
tst.run (test_do_query_and_show, tag='dummy')

### Change the metric that we want to show

In [29]:
#export tests.tools.test_query
def test_do_query_and_show_change_metric ():
    em = generate_data ('do_query_and_show_change_metric')
    
    # if we want to use a single metric, we can simply indicate its name:
    do_query_and_show (metric='test_accuracy', manager_path=em.manager_path)
    
    # we can indicate more than one metric, using a list:
    do_query_and_show (metric=['test_accuracy', 'validation_accuracy'], manager_path=em.manager_path)
    
    em.remove_previous_experiments (parent=True)

In [30]:
tst.run (test_do_query_and_show_change_metric, tag='dummy')

### Run query with conditions

In [31]:
#export tests.tools.test_query
def test_do_query_and_show_with_conditions ():
    em = generate_data ('do_query_and_show_with_conditions')
    
    do_query_and_show (metric='validation_accuracy', op='max', pf={'epochs':15}, 
                       manager_path=em.manager_path)
    
    em.remove_previous_experiments (parent=True)

In [32]:
tst.run (test_do_query_and_show_with_conditions, tag='dummy')

### Run query that sorts by maximum

In [33]:
#export tests.tools.test_query
def test_do_query_and_show_sort_maximum ():
    em = generate_data ('do_query_and_show_sort_maximum')
    
    do_query_and_show (metric='validation_accuracy', op='max', sort='max', 
                       stats=['mean', 'min', 'max'], manager_path=em.manager_path);
    
    em.remove_previous_experiments (parent=True)

In [34]:
tst.run (test_do_query_and_show_sort_maximum, tag='dummy')

## parse_args

In [35]:
#export
def parse_args(args):
    default_always = ''

    parser = argparse.ArgumentParser(description='Run query')
    # Datasets
    parser.add_argument('--stats', type=str, nargs='+', default=['mean'],  help="statistics for multiple runs")
    parser.add_argument('--experiments', type=int, nargs='+', default=None,  help="experiment numbers")
    parser.add_argument('-v', type=str, default='{}', help='variable parameters')
    parser.add_argument('-f', type=str, default='{}', help='fixed parameters')
    parser.add_argument('-a', type=str, default='[]', help='all parameters')
    parser.add_argument('-e', '--exact', action= "store_true", help='exact match') 
    parser.add_argument('--last', type=int, default=None, help='include these last experiments') 
    parser.add_argument('--best', type=int, default=None, help='include these best experiments')
    parser.add_argument('--range', type=int, nargs='+', default=None, help='include this range of experiments')
    parser.add_argument('-c', '--compact', type=int, default=0, help='compact parameters to this number of characters') 
    parser.add_argument('--results', type=int, default=0, help='min number of results to consider') 
    parser.add_argument('-s', '--show', action= "store_true")
    parser.add_argument('--other', action= "store_true")
    parser.add_argument('--always', type=str, default = default_always)
    parser.add_argument('--round', default=2, type=int, help='round scores to this number of digits')
    parser.add_argument('--runs', default=None, type=int, nargs='+', help='query restricted to run number provided')
    parser.add_argument('--sort', default=None, type=str)
    parser.add_argument('--width', default=None, type=int, help='max column width')
    parser.add_argument('--metric', type=str, nargs='+', default=None, help='include these metrics')
    add_em_args (parser, but=['metric'])
    pars = parser.parse_args(args)

    pars.v = eval(pars.v)
    pars.f = eval(pars.f)
    pars.a = eval(pars.a)
    pars.always = eval('dict(%s)' %pars.always)
    pars.f.update(pars.always)

    print (f'dictionary of query terms={pars.f}')
    
    return pars

## parse_arguments_and_query

In [36]:
#export
def parse_arguments_and_query (args):
    
    pars = parse_args(args)

    do_query_and_show (pall=pars.a, best=pars.best, compact=pars.compact, exact=pars.exact, 
                       experiments=pars.experiments, pf=pars.f, last=pars.last, 
                       other_parameters=pars.other, input_range=pars.range, results=pars.results, 
                       round=pars.round, runs=pars.runs, show=pars.show, stats=pars.stats,
                       pv=pars.v, sort=pars.sort, col_width=pars.width, **get_em_args (pars))

def main():
    parse_arguments_and_query (sys.argv[1:])

### Change the metric that we want to show

In [37]:
#export tests.tools.test_query
def test_parse_arguments_and_query_change_metric ():
    em = generate_data ('parse_arguments_and_query_change_metric')
       
    # indicate more than one metric
    command = f'--metric test_accuracy validation_accuracy --manager_path {em.manager_path}'
    parse_arguments_and_query (command.split())
    
    # indicate a single metric
    command = f'--metric test_accuracy --manager_path {em.manager_path}'
    parse_arguments_and_query (command.split())
    
    em.remove_previous_experiments (parent=True)

In [38]:
tst.run (test_parse_arguments_and_query_change_metric, tag='dummy')

### Run query with conditions

In [39]:
#export tests.tools.test_query
def test_parse_arguments_and_query_with_conditions ():
    em = generate_data ('parse_arguments_and_query_with_conditions')

    command = f'--metric validation_accuracy --op max -f dict(epochs=15) --manager_path {em.manager_path}'
    parse_arguments_and_query (command.split())

    em.remove_previous_experiments (parent=True)

In [40]:
tst.run (test_parse_arguments_and_query_with_conditions, tag='dummy')